In [12]:
import numpy as np
import pandas as pd
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

In [7]:
import sys
sys.path.insert(0, '../libs/')
%matplotlib inline

import FeatureExtraction
from lxml import etree

training_dataset = '../../dataset/training/pan12-sexual-predator-identification-training-corpus-2012-05-01.xml'
test_dataset = '../../dataset/test/pan12-sexual-predator-identification-test-corpus-2012-05-17.xml'

sexual_predator_ids_file = '../../dataset/training/pan12-sexual-predator-identification-training-corpus-predators-2012-05-01.txt'

chat_based_features_csv='../../csv/chat_based_features_training.csv'

In [3]:
training_tree = etree.parse(training_dataset)

In [4]:
autor_conversation_dictionay = FeatureExtraction.extract_author_conversation_node_dictionary_from_XML(training_tree)
sexual_predator_ids = FeatureExtraction.sexual_predator_ids(sexual_predator_ids_file)

In [8]:
frame=pd.DataFrame(pd.read_csv(chat_based_features_csv))


In [9]:
frame[[-1]]
frame[[-2,-1]]


,mean time of messages sent,is sexual predator
0,23820,0
1,20760,0
2,32040,0
3,61320,0
4,40800,0
5,60060,0
6,56460,0
7,53100,0
8,38760,0
9,5040,0


In [10]:
y=np.zeros(len(frame))
for index,row in frame.iterrows():
    if(row['autor'] in  sexual_predator_ids):
        y[index]=1
print(len(y[y==1]))
print(len(sexual_predator_ids))

142
142


In [ ]:
clf=svm.SVC(kernel='linear',cache_size=200,degree=3,shrinking=True)
to_excl=['autor','mean time of messages sent','is sexual predator']
features=['number of conversation', 'percent of conversations started by the author', 'number of messages sent', 'average percent of lines in conversation', 'number of characters sent by the author']
X=frame.ix[:,features]
y=np.ravel(frame[[-1]])
print(y[y==1])
clf.fit(X,y)